# 1. Import

In [1]:
!pip install langchain langchain-openai tiktoken openai faiss-cpu pypdf chromadb langchain_community


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 65.1 MB/s eta 0:00:00

In [2]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.schema import Document
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.chains import LLMChain

In [4]:
# llm 모델 설정
llm = ChatOpenAI(
    model_name = "gpt-4o-mini",
    temperature=0.7,
    streaming=True # 스트리밍 출력
)

In [ ]:
prompt_template = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 광고 카피라이터입니다. 주어진 카드 정보와 혜택을 바탕으로 기발하고 명확한 카피라이팅 문구를 제작하는 것이 역할입니다.
당신의 문구는 사용자에게 카드의 가치를 한눈에 전달할 수 있어야 하며, 읽는 사람이 혜택을 즉각적으로 이해하고 매력을 느끼게 해야 합니다.
이모티콘을 더해서 산뜻한 느낌을 더해주세요.
<|eot_id|><|start_header_id|>user<|end_header_id|>

카드 정보:
{llm_input}

요청사항:
입력된 카드 정보 각각에 대해 2~3개의 기발한 광고 문구를 작성해주세요.

예시 문구:
1. "영화와 캐시백의 만남 – 삼성카드 MILEAGE PLATINUM이 선사합니다!"
2. "와이즈한 선택, 국민 와이즈카드로 누리는 뷰티와 문화의 향연!"

위와 같은 스타일로 작성해주세요.
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

In [9]:
prompt_template = """


당신은 뛰어난 광고 카피라이터입니다. 주어진 카드 정보와 혜택을 바탕으로 창의적이고 감동적인 카피라이팅 문구를 제작하는 것이 역할입니다.
- 당신의 문구는 사용자에게 카드의 가치를 한눈에 전달할 수 있어야 하며, 읽는 사람이 혜택을 즉각적으로 이해하고 매력을 느끼게 해야 합니다.
- 당신의 문구는 단순한 정보 전달이 아니라, 감정적인 연결을 만들어내야 합니다.
- 사용자가 이 카드를 소유했을 때 느낄 수 있는 만족감을 생생하게 그려주세요.
- 구체적인 혜택을 중심으로 재미있고 톡톡 튀는 문구를 작성해주세요.
- 이모티콘을 더해서 산뜻하고 생동감 있는 문구를 만들어주세요.

작성 시 참고할 핵심 요소 :
1. 카드의 혜택과 특징을 한 문장으로 임팩트 있게 전달합니다.
2. 사용자에게 긍정적이고 매력적인 이미지를 상상하게 합니다.
3. 상황에 따라 "감성적인 접근" 또는 "실용적인 접근"으로 나뉘어 작성할 수 있습니다.


요청사항:
입력된 카드 정보 각각에 대해 3개의 톡톡 튀는 광고 문구를 작성해주세요.
각 문구는 다음을 포함해야 합니다:
- 카드의 주요 혜택 강조
- 감성적 또는 실용적 요소
- 이모티콘으로 생동감 추가

예시 문구:
1. "커피 한 잔의 행복도 더! ☕ 카페 할인 혜택, 바로 신한카드 SIMPLE로!"
2. "여행 가방 속 필수템, 대한항공 마일리지 카드✈️로 세계를 담아보세요."
3. "월말 지출도 든든하게! 💳 삼성카드 SAVE로 캐시백의 즐거움!"

카드 정보:
{llm_input}

작성할 문구:
1.
2.
3.

"""

prompt = PromptTemplate(input_variables=["llm_input"],template=prompt_template)

# LLM Chain 생성
llm_chain = LLMChain(llm=llm, prompt=prompt)

llm_input = """
- 카드 이름: KB국민 와이즈카드
- 혜택: 뷰티, 납부, 카페/베이커리, 쇼핑/간편결제, 의료, 교통/주유, 문화, 교육/육아
"""

response = llm_chain.run({"llm_input":llm_input})

# 결과 출력
print(response)

1. "아름다움도 스마트하게! 💖 KB국민 와이즈카드로 뷰티 할인과 함께 나만의 스타일을 완성해보세요!" 

2. "일상 속 작은 행복, KB국민 와이즈카드와 함께! ☕ 카페와 베이커리에서의 특별한 순간을 더해보세요!" 

3. "가족의 건강과 교육, 두 마리 토끼를 잡는 똑똑한 선택! 🎓💳 KB국민 와이즈카드로 세상에 꼭 필요한 혜택을 모두 누리세요!"
